#### Import Libraries

In [15]:
from transformers import pipeline, DistilBertTokenizerFast
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

#### Load Cleaned Datasets

In [16]:
# Load the cleaned datasets
train_df = pd.read_csv('E:\\Vocational\\Lighthouse Labs\\Flex Course\\Projects\\P05_Large Language Models\\llm_project\\data\\cleaned_train.csv.gz', compression='gzip')
test_df = pd.read_csv('E:\\Vocational\\Lighthouse Labs\\Flex Course\\Projects\\P05_Large Language Models\\llm_project\\data\\cleaned_test.csv.gz', compression='gzip')

# Extract labels
y_train = train_df['label']
y_test = test_df['label']

#### Initialize Sentiment Analysis Pipeline

In [17]:
# Initialize the sentiment analysis pipeline with DistilBERT
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\16476\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Tokenization and Inference

In [18]:
# Function to get predictions from the pipeline
def get_predictions(text_list, classifier, max_length=512):
    # Use the classifier pipeline directly, which handles tokenization internally
    preds = classifier(text_list, padding=True, truncation=True, max_length=max_length)
    # Extract the labels from the predictions
    pred_labels = [1 if pred['label'] == 'POSITIVE' else 0 for pred in preds]
    return pred_labels

# Get predictions for train and test datasets
train_preds = get_predictions(train_df['clean_text'].tolist(), classifier)
test_preds = get_predictions(test_df['clean_text'].tolist(), classifier)

# Verify predictions
print(train_preds[:10])
print(test_preds[:10])

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0]


#### Evaluation

In [19]:
# Evaluate the model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print('Confusion Matrix:')
    print(cm)
    print('Classification Report:')
    print(report)
    
    return accuracy, precision, recall, f1, cm, report

# Evaluate train predictions
print("Train Set Evaluation:")
evaluate_model(y_train, train_preds)

# Evaluate test predictions
print("Test Set Evaluation:")
evaluate_model(y_test, test_preds)

Train Set Evaluation:
Accuracy: 0.7956954706071314
Precision: 0.9187840290381125
Recall: 0.6494547787042976
F1 Score: 0.7609921082299888
Confusion Matrix:
[[11716   716]
 [ 4372  8100]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.94      0.82     12432
           1       0.92      0.65      0.76     12472

    accuracy                           0.80     24904
   macro avg       0.82      0.80      0.79     24904
weighted avg       0.82      0.80      0.79     24904

Test Set Evaluation:
Accuracy: 0.7987984355469537
Precision: 0.920999095840868
Recall: 0.655064308681672
F1 Score: 0.7655956407365652
Confusion Matrix:
[[11662   699]
 [ 4291  8149]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.94      0.82     12361
           1       0.92      0.66      0.77     12440

    accuracy                           0.80     24801
   macro avg       0.83      0.80      0

(0.7987984355469537,
 0.920999095840868,
 0.655064308681672,
 0.7655956407365652,
 array([[11662,   699],
        [ 4291,  8149]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       0.73      0.94      0.82     12361\n           1       0.92      0.66      0.77     12440\n\n    accuracy                           0.80     24801\n   macro avg       0.83      0.80      0.79     24801\nweighted avg       0.83      0.80      0.79     24801\n')